In [ ]:
using DataFrames
using CSV
using Glob
using Chain
using Statistics
using SimSpread


In [ ]:


path = "/home/fabian/simspread/simspread_timesplit_predictions"
files = glob("*.out", path)
df = DataFrame()


for file in files
    # extract
    α = @chain file begin
        basename
        replace(".out" => "")
        split('.')
        last
        split('_')
        last
    end
    
    α_value = if startswith(α, "0")
              parse(Float64, α, ) / 100
          else
              parse(Float64, α) / 10
          end
 
    
    # file to df
    file_df = CSV.File(file, delim=",", header=["Fold", "Ligando", "Target", "Score", "TP"]) |> DataFrame
    
    # add alpha 
    file_df[!, "Alpha"] .= α_value
    
    append!(df, file_df)
end




In [ ]:
# alpha_values = unique(df[:, "Alpha"])
# alpha_values = sort(alpha_values)



20-element Vector{Float64}:
 0.0
 0.01
 0.02
 0.03
 0.04
 0.05
 0.06
 0.07
 0.08
 0.09
 0.15
 0.25
 0.35
 0.45
 0.55
 0.65
 0.75
 0.85
 0.95
 1.0

In [15]:
df_2 = df
df_2 = @chain df_2 begin
    groupby([:Fold, :Ligando, :Target, :TP, :Alpha])    
    combine(:Score => sum => :Score)
    unstack(:Alpha, :Score)
    transform(AsTable(5:24) .=>
              ByRow.([mean, maximum, sum]) .=>
              [:Mean, :Max, :Sum])
    select([1,2,3,4,9,25,26,27])
end



Row,Fold,Ligando,Target,TP,0.02,Mean,Max,Sum
,Int64,String15,String15,Float64,Float64?,Float64?,Float64?,Float64?
1,1,CHEMBL1062,CHEMBL1075132,0.0,0.0,4.94711e-6,5.02698e-5,9.89422e-5
2,1,CHEMBL1062,CHEMBL1075145,0.0,0.0,6.18897e-7,5.72355e-6,1.23779e-5
3,1,CHEMBL1062,CHEMBL1075165,0.0,0.0,2.61921e-6,2.20103e-5,5.23842e-5
4,1,CHEMBL1062,CHEMBL1075323,0.0,0.0,1.30208e-6,9.3893e-6,2.60415e-5
5,1,CHEMBL1062,CHEMBL1163125,0.0,0.0,5.68893e-6,4.35057e-5,0.000113779
6,1,CHEMBL1062,CHEMBL1250378,0.0,0.0,3.44353e-6,5.37322e-5,6.88706e-5
7,1,CHEMBL1062,CHEMBL1293224,0.0,0.0,2.32767e-6,1.6584e-5,4.65534e-5
8,1,CHEMBL1062,CHEMBL1293226,0.0,0.0,6.49063e-6,5.93017e-5,0.000129813
9,1,CHEMBL1062,CHEMBL1293267,0.0,0.0,4.11327e-6,3.11141e-5,8.22653e-5
